In [19]:
# Setup environ
import os 
from dotenv import load_dotenv
load_dotenv()

True

# Initialize LLM

In [2]:
from langchain.llms import OpenAI

# Initialize LLM (From openai in this case)
llm = OpenAI(temperature = 0.9)

In [3]:
# Test

text = "What are the first 5 places to travel in Thailand"
output = llm(text)
print(output)



1. Bangkok – the vibrant capital of Thailand, filled with culture, nightlife, and world-renowned street food.

2. Chiang Mai – home to ancient temples, surrounded by lush nature, and a great spot for trekking.

3. Phuket – a tropical paradise with stunning beaches, coral reefs and lively nightlife.

4. Krabi – a coastal region of limestone cliffs, emerald waters, and beautiful caves.

5. Koh Samui – an idyllic island with a stunning coastline, luxurious resorts and quirky nightlife.


# Prompt Template

- offered by langchain, in case the prompt is a variable

In [4]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables = ['country'],
    template = 'What are the first 5 places to travel in {country}'
                       )

Here's the arguments of PromptTemplate

<fieldset>
input_variables: List[str] : """A list of the names of the variables the prompt template expects."""

template: str
"""The prompt template."""

template_format: str = "f-string"
"""The format of the prompt template. Options are: 'f-string', 'jinja2'."""

validate_template: bool = True
"""Whether or not to try validating the template."""
    </fieldset>

In [6]:
text = prompt.format(country='England')
print('text : ' + text)
output = llm(text)
print('output : ' + output)

text : What are the first 5 places to travel in England
output : 

1. London
2. Oxford
3. Bath
4. Norfolk
5. The Cotswolds


# Tools <- Agent -> LM

- Once the agent got prompt, they will split the prompt into chunks (chains) if there are multiple questions in 1 prompt, and figure out how to implement tool per chain. Note that the one who decided to break chain and tools to use is under LM

In [15]:
# os.environ['SERPAPI_API_KEY']
from langchain.agents import initialize_agent
from langchain.agents import load_tools
from langchain.llms import OpenAI

In [22]:
llm = OpenAI(temperature=0.0)
tools = load_tools(['serpapi','llm-math'], llm=llm)
agents = initialize_agent(tools, llm, agent='zero-shot-react-description',verbose=True)

- The reason why we need llm as arguments is because of llm-math (it depends on the type of llm)
- Without llm-math, the llm argument llm is not required
- verbose=True

In [25]:
text = 'Who is the current prime minister of Thailand, and how long he stayed as prime minister, and what is the next expected prime minister'
agents.run(text)



> Entering new AgentExecutor chain...
 I need to find out who the current prime minister is, how long he has been in office, and who is expected to be the next prime minister
Action: Search
Action Input: "Current prime minister of Thailand"
Observation: Prayut Chan-o-cha
Thought: I need to find out how long he has been in office
Action: Search
Action Input: "Prayut Chan-o-cha prime minister of Thailand"
Observation: Prayut Chan-o-cha is a Thai politician and retired army officer who has served as the Prime Minister of Thailand since he seized power in a military coup in 2014. He is concurrently the Minister of Defence, a position he has held in his own government since 2019.
Thought: I need to find out who is expected to be the next prime minister
Action: Search
Action Input: "Next expected prime minister of Thailand"
Observation: Voting is set for Sunday, with progressive parties calling for reforms to the monarchy and military.
Thought: I now know the final answer
Final Answer: Pra

'Prayut Chan-o-cha has been the Prime Minister of Thailand since 2014 and is expected to remain in office until the next election, which is set for Sunday.'

# Memory : Add state to agents and chains

The previous agent does not memorize? It is decided for a series of user prompt.

In [28]:
# Another type of agents which decided to memorize the previous prompt
from langchain import ConversationChain
llm = OpenAI(temperature=0.0)
conversation = ConversationChain(llm=llm,verbose=True)

In [30]:
conversation.predict(input='Hello, I love eating salad')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello, I love eating salad
AI:

> Finished chain.


" Hi there! That's great to hear. I'm an AI, so I don't eat food, but I do love learning about different types of food. What kind of salad do you like to eat?"

In [31]:
conversation.predict(input='Salad with mushroom')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello, I love eating salad
AI:  Hi there! That's great to hear. I'm an AI, so I don't eat food, but I do love learning about different types of food. What kind of salad do you like to eat?
Human: Salad with mushroom
AI:

> Finished chain.


' Interesting! Mushrooms are a great addition to salads. Do you like to add any other ingredients to your salads?'

In [33]:
conversation.predict(input='What did I say at the first time')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello, I love eating salad
AI:  Hi there! That's great to hear. I'm an AI, so I don't eat food, but I do love learning about different types of food. What kind of salad do you like to eat?
Human: Salad with mushroom
AI:  Interesting! Mushrooms are a great addition to salads. Do you like to add any other ingredients to your salads?
Human: Do you think I love eating meat or vegetarian
AI:  I'm not sure. It's hard to say without knowing more about your preferences. Do you usually eat meat or vegetarian dishes?
Human: What did I say at the first time
AI:

> Finished chain.


'  At the first time, you said that you love eating salad.'